This script aims at **forecasting the 2024 U.S. national corn yield (bu/acre)** using historical USDA yield data and provided weather data.

The corn data are obtained with API and downloaded automatically.

Only the category "ALL PRODUCTION PRACTICES" (see USDA agric. database website) has been considered (thus there is no difference between irrigated and non-irrigated fields). --> The model could be relaxed to actually discriminate between the two cases, as we expect different weather features to be important.

Only the counties for which the weather was available have been considered in the model. As a next step for including the other counties there is a first easy move:
We can look for the trend of each of these counties (or aggregated counties under the name of "OTHER (AGGREGATE)" under the USDA data) and then we can model the remaining fluctuations as a random walk.
We can then do some quick Monte Carlo modeling for including these counties in the forecast.
However, I expect the weight of these datasets to be small as otherwise they would not be aggregated. Furthermore, if counties are far away (especially in big states) a real modeling would be complicated as each underlying random process could mix and be undetectable.

Before developing any model I find very constructive and insightful to look at the data and perform some correlations to have a first hint. I started comparing the corn yield among different states.

The first interesting result is that the standard deviation can be extremely different among the different states. An example is **Ohio** and **South Dakota**. Thus it appeared clear that probably each state must be modeled independently to reduce the variance of the prediction. Thus I have done some weather plots for Ohio and South Dakota to see if already graphically it was easy to detect the differences. [*some weather considerations..*]

After this first investigation, I focused on **feature engineering** and **corn data transformation**.

In particular from the weather dataset I extracted some values that could be relevant for the corn yield modeling. The list of these features is in the first cell of the notebook. Some of the features are the **maximum continus days without rains**, the **max temperature in the month**, **very cold day** (T<-5 degree>) and **very hot days** (T>35 degrees) and I selected them due to the previous observations and differences between Ohio and South Dakota.

For what concerns the modeling of the corn yield, it is done essentially in a few steps. After looking at the simple plot corn yield versus time it appeared clear they have a strong trend. The first assumption here is that the trend does not depend on the weather. This is reasonable given the huge growth of corn yield cannot be justified with weather. Thus, as first thing I estimated the trend for each county.

My idea (motivated by the fact that such feature is quite common in time series) is that this growth is due to investments and technology. More corn yield leads to more investments and more than a linear trend is expected. Thus the **first transformation** that I have applied to the data is a **logarithmic** one. After the log transform there is still some trend that can has also been removed. However, to **remove the additional trend** I only took the data compatible with the weather data I had at my disposal. I have done this because forecast models work well with stationary data. Thus I wanted stationary data for the available dataset (i.e. weather dataset after 2000s) I could use to train my models.

After the computation of the aggregated features (max number of days without rain, number of hot (T>35) days etc...), I have performed **correlation** plots between corn yield and each of these features for every month. These plots revealed something extremely useful. The correlation, for each feature, is a cyclic function of time (it is obvious that correlation in December must be similar to the one in January but it is still a good safety check) and the **peak of correlation** (or anti-correlation) for almost all features is reached during **summer between May and August**. This is very clear for the plot obtained including the whole weather dataset (i.e. the data are not filtered per state or county) but it is also clear in the individual plots of single states (except Minnesota whose plot looks very noisy).

The correlation plot shows that precipitation, soil water content and soil water content standard deviation are positively correlated with corn yield during summer, while extreme temperatures (very hot days or maximum temperatures) are negatively correlated with corn yield during summer.

Unfortunately this plot tells us something important: the crucial weather data to have good forecasts are between May and August. This means that with our dataset we cannot do reliable 2024 corn yield prediction as our 2024 weather data stop at the end of April. To include this information in our data forecast we should heavily increase the error bars (or variance or confidence range) compared to the simple confidence interval we have from our modeling. In this assignment, this is not included thus we should be less confident in our forecast compared to what data tell us. One could include this assessment by training the model with the full year weather for past years and then to do the same with just the data we have to estimate our 2024 corn yield, that is data until April. Then one could compute how much is the difference for example compared to real values of 2021, 2022 and 2023 between the two models and the real corn yield value. I did not do it myself but it should be doable with this notebook modifying the parameters in the first cell. I have only done the prediction for **2023** and it looks **very good (error of prediction of 1%)** when full year is included, especially with the random forest model. This can be done putting YEAR_PREDICTION to 2023 and month range (5,8) -it is enough-.

To actualy show that the model is betteer than a simple fit, next to the prediction for 2024 there is the prediction for the year **2012**. I chose this year since it goes outsite the linear fit and it has a strong drop compared to 2011 and 2013. The random forest model is actually capable of predicting the drop as you can see from the picture. For this plot I did not retrain the model so the model has seen partially the data during trainig, but the match is nonetheless very good and it stresses that the random forrest is doing a very good job.

Let us go to the more quantitative part:

The models developed in this notebook start from data of each single county. The idea is to **model the corn yield of each county for 2024 and then to weight them** (the weight is the relative harvested area with respect to the total) for the final national corn yield prediction. Indeed from the plots drawn below it is clear that there are counties with very different areas (making some almost negligible for national corn yield).

Here there is again some space for model improvement. Indeed during the training of our models all counties are treated in the same way, one could think of a customized loss function where more importance is given to data belonging to counties with large harvested areas.

Further improvement could also be achieved with a model for the harvested areas. In this assignment I focused only on the yield, but one could apply similar modeling (using other datasets - not weather) for the harvested areas. Not all areas are available for 2024 thus we have to make some choice. There are **two possibilities** in the notebook for what concerns the **area** to use for the weight of the forecast:

1) The last available area for the counties

2) The average of the data after the year 2000 (to be coherent with the rest of the modeling)

***Thus learning process happens on this transformed dataset and then the predicted values are re-transformed*** (in the opposite order) to go back to real values. In the evaluation of the uncertainty parameters I did not include also the uncertainty of the trend, but for a correct modeling this should be done.

In this assignment I mostly focused on the modeling part, not on the efficiency of the models nor on the exploitation of the models. I am sure that given the two models here one could play with additional aggregated weather features and with some other parameters (i.e. tuning harvested areas, including other counties, changing parameters of the models and so on) to obtain more precise forecasts.

The models used to give a quantitative estimation for 2024 are two:

1) **Random Forest** (using scikit-learn)

2) **Multi Layer Perceptron** (Neural Network with Pytorch)

They are both very simple, and yet they give reasonable forecasts. Both of them could be further tuned even though I did not spend time trying to optimize the results.

As a first step to improve, the random forrest could take as input the number of trees depending by the number of input features (now it is fixed thus it is not optimal for the case with monthly aggregated data)

Since we are dealing with time series, another possible extension of this work could be the possibility of using different NN architectures that are more appropriate to model time series such as LSTM or the time-series version of Transformer. The latter could also give information about which hidden parameters are most critical thanks to the "attention" mechanism.

I have also started to implement the probabilistic multilayer perceptron, and this is something I would finish in the future - it does not differ much from the simple MLP.

Another simple option would be to perform a Bayesian regression, however it could be demanding with many dimensions (i.e. all weather features) thus I disregarded it at first.

The notebook is logically divided in three parts:

1) **Data engineering**\
In this first part weather data are loaded, the corn data are downloaded with an API (be sure the internet connection is fine and the API_KEY works), and there is some data manipulation for ordering correctly the data for the visualization and some checks between API downloaded data and manually downloaded data - the part is commented out since you need the file to load but you can still see the code.

2) **Data exploration**\
In this part there is some exploration of data done plotting several quantities to understand where and how to start the modeling. Some parameters as temperature, precipitation and soil moisture are plotted for states with very different corn yield. The goal was to see if already visually when comparing different locations with very different yield simple plots can already be inspiring. From those, one can target better subsequent analysis.\
After, there is the calculation of weather aggregated quantities. In particular I have computed: tmax_mean, tmax_min, tmax_max, tmin_mean, tmin_max, tmin_min, tmin_std, precip_max, precip_std, precip_mean, swvl2_min, swvl2_mean, swvl2_std, hot_day (days where temperature is above 35 degrees), cold_day (days where temperature is below -5), max_cons_dry_days (maximum consecutive number of days without rain).

3) **Modelling and prediction**\
In this part there is the modeling for the training algorithms. There is first the random forest, with training and prediction. Then there is the Neural Network with the training and prediction.
The random forest is trained twice and it generates two predictions. The first includes only the yearly aggregated features (but including only the months between January and April) and the second one includes monthly aggregated features (thus 4 times more data than the yearly aggregated case). The Neural Network is trained only on the monthly aggregated data. Both the models give very similar results, with a slight overestimation of the 2023 value (when the input YEAR_PREDICTION is set to 2023) of about 2%. However, training the random forest with the relevant weather dataset for July (i.e. taking weather data from May to August), the model perfectly finds the right 2023 value.
The confidence level in both models is about 8% of the mean value, i.e. data are supposed to fall inside 195 (mean value predicted 2024) ±10/15. The predicted value is very similar for both the random forest and the NN.
The NN has been trained with the data of the whole nation together, and it could be better tuned if it was trained state-by-state. Indeed with the random forest I show that the mean root square error is 0.15 when the whole data are trained together and 0.13 when it is trained state-by-state. I expect a similar improvement to occur also for the NN.






The following are input parameters that can be changed.

In [ ]:
YEAR_PREDICTION      = 2024  # year for which we want to do the forecast
                             # The training (and testing) dataset do include until   "YEAR_PREDICTION"
                             # if YEAR_PREDICTION = 2020 only data until 2019 are used for training

MONTHS_RANGE         = (1,4) # This is the interval of the year -espressed in month [begging_month, final_month]
                             # that we use to train the model and predict the results.
                             # It is (1,4) because weather data for 2024 only arrive until April

features_NN_training = "all" # This uses all possible features for the training of the NN, 
                             # the full list is listed below. As next step I would surely allow the user to 
                             # specify only a subset of it. I tried to keep the code generic enough to do it, but I have never tested
                             # if a subset of feature runs top-bottom.

AREA_AVERAGED        = False  # This parameter is to define what to do with the harvested areas.
                              # True  --> the area is the averaged (within a county) area over the last 20 years.
                              # False --> the area is the most recent available area (for each county).

                              # Indeed while there is clear an increasing trend for corn yield the harvested area is more noisi


API_KEY              = "" # Insert here the API key to connect to the USA government website and download the data. 
                                                              


RANDOM_FOREST = False

NN_MLP_STATE  = False # logical not yet implemented


"""
to select only few use e.g. features_NN_training = ['tmax_mean', 'tmax_min', 'tmax_max']
['tmax_mean', 'tmax_min', 'tmax_max','tmin_mean', 
 'tmin_max', 'tmin_min', 'tmin_std', 'precip_max',
 'precip_std', 'precip_mean', 'swvl2_min', 'swvl2_mean', 'swvl2_std',
       'hot_day', 'cold_day', 'max_cons_dry_days']
"""


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
wheater_df = pd.read_parquet("hist_wx_df.parquet");

wheater_df = wheater_df.sort_values(by=["adm1_name","adm2_name","date"]);
wheater_df.head(1)

states   = wheater_df['adm1_name'].unique().tolist()      # Unique state names
places = dict()
for state in states:
    tmp = wheater_df[wheater_df["adm1_name"]==state]['adm2_name'].unique().tolist()
    places[state] = tmp
    
places.keys()

Now it is crucial to see which data we have available for a prediction of 2024. 
We have until the **last day** in hist_wx_df dataframe, that is we have to exploit weather data until the end of April.

We still use all our data available because it can be useful to see how much data only available until end of April under-perform compared to full data (until harvest time)

In [ ]:
def interpolate_group(df):
    df = df.set_index('date').asfreq('D')
    df[['adm1_name', 'adm2_name']] = df[['adm1_name', 'adm2_name']].ffill().bfill()
    numeric_cols = df.select_dtypes(include='number').columns
    df[numeric_cols] = df[numeric_cols].interpolate()
    return df

wx_filled      = wheater_df.groupby(['adm1_name', 'adm2_name'], group_keys=False).apply(interpolate_group).reset_index()
wx_filled["year"]    = wx_filled["date"].dt.year
wx_filled["month"]   = wx_filled["date"].dt.to_period("M")
wx_filled['month_N'] = wx_filled['date'].dt.month

Now it is time to check that all weather data belong to the same period, i.e. the first and the last day is always the same accross all counties. 

To check this, we create a **dictionary** with keys corresponding to the first and last available day for the weather. Inside each key we have a list of all counties that have those days weather data. The format is **"county-state"** (e.g. ) ['Will-Illinois', 'Williamson-Illinois', ...]

In [ ]:
from collections import defaultdict
time_window = [defaultdict(list), defaultdict(list)]

grouped = wx_filled.groupby(['adm1_name', 'adm2_name'])['date']
for state in places.keys():
    for county in places[state]:
        dates = grouped.get_group((state, county)).sort_values()
        first_day = dates.iloc[0]
        last_day = dates.iloc[-1]

        time_window[0][first_day].append(f"{county}-{state}")
        time_window[1][last_day].append(f"{county}-{state}")


time_window is a a list of 2 dictionary. 

The key of the first dictionary <time_window[0]> is the initial date available for every city and inside the key there is the list of couties with that initial date

The key of the second dictionary <time_window[1]> is the final date available for every city and inside the key there is the list of couties with that final date

We print the output of the dictionaries to spot anomalies (We can see that **'Monroe-Kentucky'** has a shorter time window database for the weather, first day available being **2011-01-01**)

In [ ]:
for k in time_window[0]:
    print(k, time_window[0][k])
print("---------------------------")
for k in time_window[1]:
    print(k, time_window[1][k])

oldest_time = min(time_window[0].keys())
newest_time = max(time_window[0].keys())

Rather than downloading manually all the files we need we can download them using the **API_KEY** provided by the USA government.

In [ ]:
import requests
def download_corn_data(state="none", county="none",Area_harvsted=False):

    if state == "none":
        level = "NATIONAL"
    else:
        level = "COUNTY"
    

    params = {
        "key": API_KEY,
        "source_desc": "SURVEY",
        "commodity_desc": "CORN",
        "statisticcat_desc": "YIELD",
        "unit_desc": "BU / ACRE",
        "agg_level_desc": level,
        "freq_desc": "ANNUAL",
        "reference_period_desc": "YEAR",      
        "year__GE": 1830,
        "year__LE": 2024,
        "format": "CSV"
    }

    if Area_harvsted:
        params["statisticcat_desc"] = "AREA HARVESTED"
        params["unit_desc"] = "ACRES"
        params["util_practice_desc"] = "GRAIN"  # <-- aggiungere anche qui
               
    if not state == "none":
        params["state_name"] = state.upper()
    if not county == "none":
        params["county_name"] = county.upper()

    url = "https://quickstats.nass.usda.gov/api/api_GET/"
    r = requests.get(url, params=params)

    with open("corn_yield_1950_2024.csv", "wb") as f:
        f.write(r.content)

    df = pd.read_csv("corn_yield_1950_2024.csv")
    df.rename(columns={"county_name": "adm2_name", "state_name": "adm1_name"}, inplace=True)
    try:
        df["adm2_name"] = df["adm2_name"].str.title()
        df["adm1_name"]  = df["adm1_name"].str.title()
    except:
        print("using national data no state_name or county_name")

    if Area_harvsted:
        df.rename(columns={"Value": "Area"}, inplace=True)
        df["Area"] = pd.to_numeric(df["Area"].str.replace(",", ""), errors="coerce")

    return df

Now we want to create a function that allows to perform plot on quantities on random or specific counties. We will use it for explorative plot analisysis

In [ ]:
import random
def random_plot(df_list, y_axis=["Value"], year_filter=2000, marker="",location="rand",ax=None,Verbose=False):
    if not isinstance(df_list, list):
        df_list = [df_list]
    if not isinstance(y_axis, list):
        y_axis = [y_axis] * len(df_list)
    if not isinstance(marker, list):
        marker = [marker] * len(df_list)
    elif len(marker) < len(df_list):
        marker = [marker[0]] * len(df_list)
        print("list marker too short compared to number of dataset - set first value of marker for all df")

    df = df_list[0]
    state_loc_list    = df["adm1_name"].unique()
    
    if location == "rand": 
        while (True):
            N_LOC_RANDOM      = random.randint(0,len(state_loc_list)-1)
            state             = state_loc_list[N_LOC_RANDOM]
            counties_loc      = df[df["adm1_name"]==state]["adm2_name"].unique()
            county            = counties_loc[random.randint(0,len(counties_loc)-1)]
            if not("Other (Combined)" in county):
                break
    else:
        state  = location[0]
        county = location[1]


    if ax==None:
        fig, ax = plt.subplots(ncols=len(y_axis),nrows=1,figsize=(6,2))

    if len(y_axis) > 1:
        ax=ax.flatten()
    g_ax = ax

    for i_df, df in enumerate(df_list):
        mask = (df["adm1_name"] == state) & (df["adm2_name"] == county ) &  (df["year"] >= year_filter ) 

        for idx, val_plot in enumerate(y_axis):
            if len(y_axis)>1:
                g_ax = ax[idx]

            df[mask].plot(x="year",y=val_plot,ax=g_ax,marker=marker[i_df])
    

    if Verbose:
        print(county, state, year_filter)
    return (state,county)

To be positive about the right way to call the API we plot the data from the manually downloaded file versus those we obtain we API. We try with national and Indiana state data.
( I have done the test on my CV but here to run you would need the local .csv file thus I have commented this part of the code)

In [ ]:
"""
corn_indiana  = pd.read_csv("C4A8B611-247C-3C5C-8F5E-36956CBC3450-Indiana.csv")
corn_indiana.rename(columns={"County": "adm2_name", "State": "adm1_name", "Year":"year"}, inplace=True)

df            = download_corn_data("Indiana")
df            = df.sort_values(by=["adm2_name","year"])
corn_indiana  = corn_indiana.sort_values(by=["adm2_name","year"])

corn_indiana["adm2_name"] = corn_indiana["adm2_name"].str.title()
corn_indiana["adm1_name"]  = corn_indiana["adm1_name"].str.title()

fig, ax = plt.subplots(ncols=2,nrows=1,figsize=(6,2))
random_plot([df, corn_indiana], marker=["*",""],  location=("Indiana", "Benton"),ax=ax[0])
random_plot([df, corn_indiana], marker=["*",""],year_filter=0, location=("Indiana", "Benton"),ax=ax[1])
"""

The following block is used to download the corn data, it aggregates harvested area with corn yield

In [ ]:
corn_value_tot = []
for state in states:
    print(state)
    corn_df = download_corn_data(state)
    area_df = download_corn_data(state=state ,Area_harvsted=True)
    
    mask = (
        corn_df["prodn_practice_desc"].str.contains("ALL PRODUCTION PRACTICES", na=False) &
        ~corn_df["adm2_name"].str.contains(r"Other.*Counties", na=False, regex=True)
    )
    mask_area = (
        area_df["prodn_practice_desc"].str.contains("ALL PRODUCTION PRACTICES", na=False) &
        ~area_df["adm2_name"].str.contains(r"Other.*Counties", na=False, regex=True)
    )

    corn_value = corn_df.loc[mask,      ['adm2_name', 'adm1_name', 'year', 'Value']]
    area_value = area_df.loc[mask_area, ['adm2_name', 'adm1_name', 'year', 'Area']]

    # corn_value_merged = corn_value.copy()
    corn_value_merged = corn_value.merge(area_value, on=['adm2_name', 'adm1_name','year'],how="left")
    
    if len(corn_value_tot) == 0:
        corn_value_tot = corn_value_merged.copy()
    else:
        corn_value_tot = pd.concat([corn_value_tot, corn_value_merged], ignore_index=True)
    

This block is used to extract the harvested area that will be used to weight the single county yield in order to get the national one

In [ ]:
if AREA_AVERAGED:
    mask = corn_value_tot["year"].between(2000, 3000)
    area_df_pred  = corn_value_tot[mask].groupby(['adm2_name', 'adm1_name']).agg({'Area': ['mean', 'std']})
    new_columns = ['_'.join(col) for col in agg.columns]
    area_df_pred.columns = new_columns
    area_df_pred.reset_index(inplace=True)
    area_df_pred.rename(columns={"Area_mean": "Area_prediction"}, inplace=True)
else:
    idx = corn_value_tot.groupby(['adm2_name', 'adm1_name'])['year'].idxmax()
    area_df_pred = corn_value_tot.loc[idx, ['adm2_name', 'adm1_name', 'Area']].reset_index(drop=True)
    area_df_pred.rename(columns={"Area": "Area_prediction"}, inplace=True)


We try to explore da corn production data to see how they look and if there is some immediate feature we can see

In [ ]:
fig,  ax      = plt.subplots(ncols=4,nrows=1,figsize=(15,3))
fig2, ax_area = plt.subplots(ncols=4,nrows=1,figsize=(15,3))

for idx in range(len(ax)):
    loc1 = random_plot(corn_value_tot,year_filter=1900,ax=ax[idx])
    loc2 = random_plot(corn_value_tot,year_filter=1900,ax=ax[idx])
    ax[idx].legend([loc1, loc2])

    random_plot(corn_value_tot,y_axis="Area",year_filter=1900,ax=ax_area[idx])
    random_plot(corn_value_tot,y_axis="Area",year_filter=2010,ax=ax_area[idx])
    ax_area[idx].legend([loc1, loc2])

From the plots above we can see that in many couties the corn yield has been growing in time since after 1940. Thus we include this trend in our modeling.

In particular we will remove the trend since it cannot depend on weather condition (as they are mostly cyclic) and cannot alone explain the strong trend but rather the fluctuations.

Thus, after de-trending we can try to model how weather does affect yearly fluctuations.

Before modeling we want to see how data are correlated and a first insight is given by the standard deviation that we plot inside each state for every year.

In [ ]:
fig, ax = plt.subplots(ncols=4, nrows=1, figsize=(14, 4))
grouped = corn_value_tot.groupby("adm1_name")
ii = -1
for key_state, state_df in grouped:
    ii = ii + 1
    std_df  = state_df.groupby("year", as_index=False)["Value"].std()
    mean_df = state_df.groupby("year", as_index=False)["Value"].mean()
    mean_state = mean_df["Value"].to_numpy()
    year = std_df["year"].to_numpy()
    std_state = std_df["Value"].to_numpy()
    if ii < len(grouped)/2:
        ax[0].plot(year, std_state,label=key_state)    
        ax[1].plot(year, mean_state,label=key_state)
    else:
        ax[2].plot(year, std_state,label=key_state)    
        ax[3].plot(year, mean_state,label=key_state)
for ii in range(4):
    ax[ii].legend()
    if np.mod(ii,2)==0:
        ax[ii].set_title("std")
    else:
        ax[ii].set_title("mean")

It is clear that the standard deviations are growing with time and also their time-fluctuations increase. 

The fact that also std has a trend may suggest that std is not only a function of weather -that shoud have no such a strong trend- but rather depends on local (county) administration and local technology.

It thus seems reasonable and better for our pourposes to detrend data county by county to eliminate the effects due to non-random factors.

It is furthermore important to note that states with higher std fluctuation are not necessarely those with larger mean. In particular it appears Kansas and South Dakota have huge fuctuations compared to Iowa and Ohio (we split in two groups the plot for a better readibility of the plot)

Let us start with a visual analysis of how  weather can impact on production. For a plot to carry meanigfull information we must compare states with very different std and mean corn yield. Thus, according to the previous plot we choose **Ohio** and **South Dakota**. We plot as function of time (limited to the years where there is a big difference in std and mean of corn yield), the daily mean (accross the state) of **Tmin**, the daily mean (accross the state) of the **precipitations** and the daily mean (accross the state) of the **swvl1**.

In [ ]:
daily_df = wx_filled.groupby(["adm1_name", "date"]).agg({"tmin": ["std", "mean"],"precip": ["mean", "std"], "swvl1":["mean","std"],"swvl2":["mean","std"]}).reset_index()
fig, ax = plt.subplots(ncols=2,nrows=4,figsize=(18,8.5))
ax=ax.flatten()
plt.subplots_adjust(hspace=0.8, wspace=0.2)

for key_state, state_df in daily_df.groupby(["adm1_name"]):
    if key_state[0] in ["Ohio","South Dakota"]:
        mask       = (state_df["date"] >= "2000-01-01") & (state_df["date"] <= "2006-12-31")
        state_df[mask].plot(x="date",y=("tmin","mean"),label=key_state[0],ax=ax[0])
        state_df[mask].plot(x="date",y=("tmin","std"),label=key_state[0],ax=ax[1])
        state_df[mask].plot(x="date",y=("precip","mean"),label=key_state[0],ax=ax[2])
        state_df[mask].plot(x="date",y=("precip","std"),label=key_state[0],ax=ax[3])
        state_df[mask].plot(x="date",y=("swvl1","mean"),label=key_state[0],ax=ax[4])
        state_df[mask].plot(x="date",y=("swvl1","std"),label=key_state[0],ax=ax[5])
        state_df[mask].plot(x="date",y=("swvl2","mean"),label=key_state[0],ax=ax[6])
        state_df[mask].plot(x="date",y=("swvl2","std"),label=key_state[0],ax=ax[7])

ax[0].set_title("Mean tmin")
ax[1].set_title("std tmin")
ax[2].set_title("mean precip")
ax[3].set_title("std pricip")
ax[4].set_title("mean swvl1")
ax[5].set_title("std swvl1")

The images are pretty clear showing large differences between Ohio and South Dakota for all parameters. One important parameter is it swvl -**soil water content**-. First we see that swv1 and swvl2 are stronlgy correlated, thus in a first approximation we skip the analysis of swvl2. **South Dakota** has a huge std compared to Ohio, and perhaps this may explain the huge std in the corn production.

We look now to a cleaner dataset obtained averaging as before but also accross every single month.

In [ ]:
monthly = wx_filled.groupby(["adm1_name", "month"]).agg({"tmin": ["mean", "min"],"tmax": ["mean", "max"]}).reset_index()

fig, ax = plt.subplots(ncols=2,nrows=2,figsize=(18,5.8))
ax=ax.flatten()
plt.subplots_adjust(hspace=0.8, wspace=0.2)

for key_state, state_df in monthly.groupby(["adm1_name"]):
    if key_state[0] in ["Ohio","South Dakota"]:
        mask       = (state_df["month"] >= "2000-01") & (state_df["month"] <= "2006-12")
        state_df[mask].plot(x="month",y=("tmin","mean"),label=key_state[0],ax=ax[0])
        state_df[mask].plot(x="month",y=("tmin","min"),label=key_state[0],ax=ax[1])
        state_df[mask].plot(x="month",y=("tmax","mean"),label=key_state[0],ax=ax[2])
        state_df[mask].plot(x="month",y=("tmax","max"),label=key_state[0],ax=ax[3])

ax[0].set_title("Mean tmin")
ax[1].set_title("min tmin")
ax[2].set_title("mean tmax")
ax[3].set_title("max tmax")

The results are very clear: in particular the extremes of temperatures are very different and more extreme for South Dakota. Temperature minima values (thus the minimum temperature for each month) of South Dakota are always lower than those of Ohio. Similarly, during summer maxima values of temperatures (thus the maximum temperature for each month) of South Dakota are always higher than those of Ohio. These huge thermal differences might be one of the reasons behind the volatility of South Dakota corn yield.

For a meaninfull analysis raw datas are not enough, here we build aggregate data that could be usefull for corn yield modeling.
We construct a Dataset over a period  (moths or years). \
**The dataset includes:** \
Tmax mean, Tmax max, Tmin mean, Tmin min, total precip (sum), mean of precip, mean and sum of swvl1, maximum consecutive days without rain, number of hot (T>35) and cold (T<-10) days

In [ ]:
def featuring_engineering(months_range, period, state):

    wx_season = wx_filled[wx_filled['month_N'].between(months_range[0], months_range[1])].copy()
    if not(state[0] == "all"):
        wx_season = wx_season[wx_season['adm1_name'].isin(state)]


    #  Aggregate weather features
    agg         =   wx_season.groupby(['adm2_name', 'adm1_name', period]).agg({'tmax': ['mean','min', 'max'],
                                                                               'tmin': ['mean', 'max', 'min','std'],'precip': ['max','std','mean'],
                                                                               "swvl2":['min','mean','std']}) 
    agg.columns = ['_'.join(col) for col in agg.columns] # To avoind tuples and sub-coloumns
    agg         = agg.reset_index()

    #  Add hot day count
    wx_season.loc[:, 'hot_day'] = wx_season['tmax'] > 35
    hot_days = wx_season.groupby(['adm2_name', 'adm1_name', period])['hot_day'].sum().reset_index()
    agg      = agg.merge(hot_days, on=['adm2_name', 'adm1_name', period], how='left')

    #  Add cold day count
    wx_season.loc[:, 'cold_day'] = wx_season['tmin'] < -4 
    cold_days = wx_season.groupby(['adm2_name', 'adm1_name', period])['cold_day'].sum().reset_index()
    agg      = agg.merge(cold_days, on=['adm2_name', 'adm1_name', period], how='left')

    dry_records = []
    for id_group, group in wx_season.groupby(['adm2_name', 'adm1_name', period]):
        dry = group['precip'] == 0
        max_cons_days_dry    = dry.astype(int).groupby((~dry).cumsum()).sum().max()
        dry_records.append((*id_group, max_cons_days_dry))

    dry_df = pd.DataFrame(dry_records, columns=['adm2_name', 'adm1_name', period, 'max_cons_dry_days'])
    agg = agg.merge(dry_df, on=['adm2_name', 'adm1_name', period], how='left')

    if period == "month":
        agg["year"]    = agg["month"].dt.year
        agg['month_N'] = agg['month'].dt.month


    return agg

def extract_feature_list(df):
    features_wanted = [c for c in df.columns if c not in ['adm2_name', 'adm1_name','Value','month','year']]
    return features_wanted

We now build a function for the detrending of corn yield. We use statsmodels.

A priori one should find the good rule of the trend, that might not be the same for every county and most importantly it will be likely different for every state. However, to keep it simple here we perform the most common detrending techniques.
We start first we a logarithmic transform and then we apply a simple regression with statsmodels. We could also normalize the data around the mean and std 1, but it turned out to be not always necessary, it depends on the model.\
Thus the function has three boolean inputs: log_detrend, trend_detrend, normalize_data_mean_std. If activated, the transformation order is: #1 log, #2 regression, #3 mean-std normalization 

In [ ]:
import statsmodels.api as sm
def detrending_fun_statmod(log_detrend, trend_detrend, normalize_data_mean_std):
    not_included_groups = []
    model_trend_dict = {}
    for (adm1, adm2), group in corn_value_tot.groupby(["adm1_name", "adm2_name"]):
        if not(adm2 in places[adm1]):
            continue
        group = group[group["year"] >= 2000]
        y = group["Value"].to_numpy()
        
        if np.shape(y)[0]<2:
            corn_value_tot.loc[group.index, "Value-detrand"] = np.nan
            not_included_groups.append([adm1, adm2])
            continue     

        if log_detrend:
          y = np.log(y+1)

        # x=x.reshape(-1, 1)
        trend     = 0
        residual  = y  

        if trend_detrend:
          x = sm.add_constant(group["year"].to_numpy())
          model = sm.OLS(y, x).fit()
          trend    =  model.predict(x)
          residual = y - trend

        mean      = y.mean()
        std       = y.std()     

        if normalize_data_mean_std:
          residual = (y - mean) / std       
        
        corn_value_tot.loc[group.index, "Value-detrand"] = residual
        # corn_value_tot.loc[group.index, "trend"] = trend
        corn_value_tot.loc[group.index, "std"] = std
        corn_value_tot.loc[group.index, "mean"] = mean
        model_trend_dict[(adm1, adm2)] = model

    return not_included_groups, model_trend_dict

def restore_trend(df, model_trend_dict, log_detrend, trend_detrend, normalize_data_mean_std):
    restored_values = []    
    for (adm1, adm2), group in df.groupby(["adm1_name", "adm2_name"]):        
        group = group[group["year"] >= 2000].copy()
        y = group["Value_predicted"].to_numpy()
        if len(y) == 0:
            print("length is 0")
            continue

        if normalize_data_mean_std:
            std  = group["std"].to_numpy()
            mean = group["mean"].to_numpy()
            y    = y * std + mean

        if trend_detrend:
            model = model_trend_dict.get((adm1, adm2), None)
            if model is None:
                continue  

            x_raw   = group["year"].to_numpy().reshape(-1, 1) # needed for compativility with the model in case of single x-y for the group
            x_years = sm.add_constant(x_raw, has_constant='add') # needed for compativility with the model in case of single x-y for the group
            trend   = model.predict(x_years)
            y       = y + trend

        if log_detrend:
            y = np.exp(y) - 1

        restored_values.append(pd.DataFrame({"index": group.index,"Value_restored": y}))

    restored_df = pd.concat(restored_values).set_index("index")
    df.loc[restored_df.index, "Value_restored"] = restored_df["Value_restored"]

    return df 

Since the dataset has some holes and it is not always possible to handle tranformation (for example detranding), we create a funtion that removes the NaN indices.

In [ ]:
def exclude_nan_dataset(df,feature="Value"):
    
    df[feature].isna().any()
    mask_nonan = ~df[feature].isna()

    no_nan_dataset = df[mask_nonan]
    if no_nan_dataset[feature].isna().any():
        print("there are still bad data with NaN values")

    return no_nan_dataset

Now before any prediction model we want to remove the trends for data.. We compute county-by-county trends.

In [ ]:
log_transf     = True
linear_detrand = True
mean_std_norm  = False

corn_value_tot                     = exclude_nan_dataset(corn_value_tot,feature="Value")

excluded_dataset, model_trend_dict = detrending_fun_statmod(log_detrend=log_transf, trend_detrend=linear_detrand, normalize_data_mean_std=mean_std_norm)

corn_value_tot                     = exclude_nan_dataset(corn_value_tot,feature="Value-detrand")

corn_value_tot                     = exclude_nan_dataset(corn_value_tot,feature="Area")

random_plot([corn_value_tot],y_axis=["Value","Value-detrand"])


In [ ]:
# intermediate function to build the features and aggegate with con Value and harvested area
def featuring_training_dataset(months_range=(3,7),period="year",state=["all"]):
    feat_year               = featuring_engineering(months_range, period, state)
    # feat_year has the same cities available for every year, 960 ("adm1_name", "adm2_name") different
    train_yield_feat_year  = feat_year.merge(corn_value_tot, on=['adm2_name', 'adm1_name', 'year'], how='left')
    train_yield_feat_year  = train_yield_feat_year.merge(area_df_pred,   on=['adm2_name', 'adm1_name'],         how='left')

    train_yield_feat_year  = exclude_nan_dataset(train_yield_feat_year,feature="Value-detrand")

    return feat_year, train_yield_feat_year

Now we try to look for important parameters for the modeling. The simplest idea is see which are the most correlated parameters. Since correlation may change during the year, the plot is a function of the month of the year.

In [ ]:
def correlation_plot(df, features, label,ax=None):
    partial = df[features]
    corr_matrix = []
    for ii in range(df["month_N"].min(), df["month_N"].max()):
        mask = partial['month_N'] == ii
        df_corr = partial[mask].corr()[label]
        corr_values = [df_corr.get(f, np.nan) for f in features[:-1]]  # exclude target
        corr_matrix.append(corr_values)

    corr_matrix = np.array(corr_matrix)  # shape: (12, num_features)
    if ax==None:
        fig, ax = plt.subplots(figsize=(5, 3))  # note the 's' in 'subplots'

    for i, var in enumerate(features[:-1]):
        ax.plot(range(df["month_N"].min(), df["month_N"].max()), corr_matrix[:, i], label=var)

    ax.set_xlabel("Month")
    ax.set_ylabel("Correlation with Value-detrand")
    ax.legend(fontsize=6, loc='best', ncol=2)


We first load the data, then we plot the correlation for the whole dataset and then we do the same for each state.

In [ ]:
# In this cell we build the dataset of feature, we keep it alone since it takes time to run
feat_month, train_yield_feat_month = featuring_training_dataset(months_range=(1,12),period="month")
label    = "Value-detrand"
features = ['tmin_min','precip_mean','hot_day', 'swvl2_mean','cold_day', 'max_cons_dry_days','swvl2_std','month_N','tmax_mean', 'tmax_max','tmin_mean']
features.append(label)
correlation_plot(train_yield_feat_month, features, label)
feat_month_copy = feat_month.copy()
train_yield_feat_month_copy = train_yield_feat_month.copy() 

As it is clear during the several months the several features may change the way they affect the corn yield. In particular it is  clear that the crucial phase is around July (month #7), with highest values of correations (and anticorrelations).

Higher precipitations and soil water content during summer seem to be beneficial for corn fields.

Furthermore, extreme temperature are anticorrelated with corn yield.

The positive correlation does not mean that incresing indefinetively the precipitation and soil water content during summer will always be beneficial (beyond a certain threshold they will start to be anticorrelated as dependence is probably non monotonic). The same argument is valid for negative correlated features. This trend only must be interpreted with respect to the average values in the dataset and it is true only for this dataset, it is not a universal feature.

However, some important metric for forecasting can be extracted. This plot may help us to tune better our models. 

Finally, since our weather dataset for 2024 only arrive until April 2024, it is unlikely to perform a good prediction for 2024 as we indeed have seen that is June-July the period where the features are mostly correlated with the corn production.

In [ ]:
label    = "Value-detrand"
features = ['tmin_min','precip_mean','hot_day', 'swvl2_mean','cold_day', 'max_cons_dry_days','swvl2_std','month_N','tmax_mean', 'tmax_max','tmin_mean']
features.append(label)

fig, ax = plt.subplots(ncols=3,nrows=5,figsize=(20,20))
ax=ax.flatten()
plt.subplots_adjust(hspace=0.4, wspace=0.4)
id_ax = -1
for group_idx, group in train_yield_feat_month.groupby("adm1_name"):
    id_ax += 1
    correlation_plot(group, features, label,ax=ax[id_ax])
    ax[id_ax].set_title(group_idx)

The state-by-state plot shows also similar trend to the one analized using the full database, but the trends are more noise (likely due to fewer datas).

Now we try to predict the corn yield with the help on a random forest (sklearn). In the following cell we build the model.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def random_forest_model(df_training, features,plot_y_test=False):

       X = df_training[features]
       y = df_training["Value-detrand"]

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

       test_dataset  = df_training.loc[X_test.index,  ["year", "adm2_name", "adm1_name","std","mean","Area_prediction","Value"]]

       # max_depth=7 has been found after a "fine tuning"
       model =  RandomForestRegressor(n_estimators=200,  max_depth=7, bootstrap=True)
       model.fit(X_train, y_train)
       pred_bootstrap  = model.predict(X_test)

       # Performance
       rmse_bootstrap = np.sqrt(mean_squared_error(y_test, pred_bootstrap))

       test_dataset['Value_predicted'] = pred_bootstrap
       test_dataset['actual_value']    = y_test
       test_dataset.sort_values(by=["adm1_name", "adm2_name","year"],inplace=True)

       test_dataset = restore_trend(test_dataset,model_trend_dict,log_detrend=log_transf, trend_detrend=linear_detrand, normalize_data_mean_std=mean_std_norm)

       if plot_y_test:
              fig, ax = plt.subplots(ncols=2,nrows=1,figsize=(6,2))
              loc = random_plot(test_dataset   , y_axis="Value_predicted",marker='*',ax=ax[0])
              _   = random_plot(df_training , y_axis="Value-detrand", marker='*',location=loc,ax=ax[0])
              random_plot(test_dataset, y_axis="Value_restored", marker='*',location=loc,ax=ax[1])
              random_plot(df_training , y_axis="Value",          marker='*',location=loc,ax=ax[1])

       area_loc  = test_dataset["Area_prediction"].to_numpy()
       error_loc = test_dataset["Value_restored"].to_numpy() - test_dataset["Value"].to_numpy()

       # std_RF    = residuals.std(ddof=1)       

       return rmse_bootstrap, len(y_test), model, error_loc, area_loc

To see how the model perform we make a plot for a random county (to be inserted into the call to the function **plot_y_test=True**). We are still working with corn yield without trend.

To compute National yield we need to weight the yield on every county. Indeed if the yield are comparable but the area are very different the impact on national level will bu much higher for the county with big area. 

**ASSUMPION** : We assume no particular modeling at this stage for the Area for 2024. We assume it to be known. In a future stage of the modelling, the same techniques applied for corn yield could be applied to the harvested area. It might be the case that harvested area does correspond to planting area and if this is the case then we already have the data since by April (end of our weather database) the planting has already been done. To be sure we should check if there are differences between harvested and planted area.

In the following we define the function for the RF training and for the RF prediction

In [ ]:
def random_forest_training(train_yield_feat_year, features,National=False,plot_y_test=False):
    mask_time     = train_yield_feat_year["year"]<YEAR_PREDICTION
    df_training   = train_yield_feat_year[mask_time]
    model_dict = {}
    std_RF = 0
    if National:
        rmse_total, N_test, model, error_tot, area_tot = random_forest_model(df_training,features,plot_y_test=plot_y_test)
        model_dict["national"] = model
    else:
        mse_list       = []
        N_test_total   = 0
        area_list      = []
        error_list = []

        for state_loc in df_training["adm1_name"].unique():
            # train the model
            rmse, N_test, model, error_loc, area_loc = random_forest_model(df_training[df_training["adm1_name"]==state_loc], features,plot_y_test=plot_y_test)


            if error_loc.shape[0]>0:
                error_list.append(error_loc)
                area_list.append(area_loc)

            model_dict[state_loc] = model

            # evaluate the global performance for USA
            mse_list.append( N_test * rmse**2)
            N_test_total = N_test + N_test_total

        error_conc    = np.concatenate(error_list)
        area_conc     = np.concatenate(area_list)

        std_RF        = np.sqrt(np.sum((error_conc*area_conc)**2)/np.sum( area_conc**2 )) # first definition

        rmse_total = np.sqrt(1/N_test_total * np.sum(np.array(mse_list)))

    return rmse_total, model_dict, std_RF


def RF_prediction(model_dict,df_prediction,features):

    total_production = 0
    total_area       = 0
    
    for state_loc in df_prediction["adm1_name"].unique():
        X_data     = df_prediction[df_prediction["adm1_name"]==state_loc][features]
        if X_data.shape[0] > 0:

            model = model_dict[state_loc]

            prediction     = model.predict(X_data)

            # rebuild the dataframe for re-tranding 
            pred_df  = df_prediction.loc[X_data.index,  ["year", "adm2_name", "adm1_name","std","mean","Area_prediction"]]
            pred_df['Value_predicted'] = prediction

            # restore the trend of the data
            pred_df = restore_trend(pred_df,model_trend_dict,log_detrend=log_transf, trend_detrend=linear_detrand, normalize_data_mean_std=mean_std_norm)

            df_prediction.loc[X_data.index, "predicted_value"] = pred_df["Value_restored"]

            total_production = total_production + np.sum(df_prediction.loc[X_data.index, "predicted_value"].to_numpy() * df_prediction.loc[X_data.index, "Area_prediction"].to_numpy())
            total_area       = total_area + np.sum(df_prediction.loc[X_data.index, "Area_prediction"].to_numpy())

        else:
            print("No values in input")
            
    corn_yield_tot = total_production/total_area

    return df_prediction, corn_yield_tot


In the following cell we train the RF for the total dataset (without splitting between states) and evaluate the RMSE of this model.

In [ ]:
if RANDOM_FOREST:
    # training bloc
    features = ['tmin_min','precip_mean','hot_day', 'swvl2_mean','cold_day', 'max_cons_dry_days','swvl2_std','tmax_mean', 'tmax_max','tmin_mean']

    feat_year, train_yield_feat_year  = featuring_training_dataset(months_range=MONTHS_RANGE)
    rmse_total, model_dict, _    = random_forest_training(train_yield_feat_year, features, National=True, plot_y_test=False)

    print(rmse_total)

Now we do the same by the training happen state-by-state and it produces a dictionary with the several models that will be used later for forecasting

In [ ]:
if RANDOM_FOREST:
    # training bloc
    features = ['tmin_min','precip_mean','hot_day', 'swvl2_mean','cold_day', 'max_cons_dry_days','swvl2_std','tmax_mean', 'tmax_max','tmin_mean']

    feat_year, train_yield_feat_year  = featuring_training_dataset(months_range=MONTHS_RANGE)
    rmse_total, model_dict, std_RF    = random_forest_training(train_yield_feat_year, features, plot_y_test=False)
    print(rmse_total)
    print("standard deviation is: "+str(std_RF))

We indeed observe that RMSE for the state-by-state model is better than the model trained on the full dataset. We will use the state-by-state model for forecast.

As we can see, the training is more performant when executed for each state undependently, with root mean square error that is 0.155 when all data trained together and 0.132 when the model is trained state by state. Thus for the prediction we use the state-by-state trained model.

Before looking at the prevision for the year 2024, let us see how much is the standard deviation for our model. It is computed assuming that the validation set is rapresentative of what could happen in 2024

In [ ]:
# prediction block
def prediction(feat_year, year,features):

    df_prediction    = feat_year[feat_year["year"]==year]
    corn_stats       = (corn_value_tot.drop_duplicates(subset=["adm1_name", "adm2_name"])[["adm1_name", "adm2_name", "mean", "std"]])

    df_prediction    = df_prediction.merge(corn_stats, on=['adm2_name', 'adm1_name'], how='inner') # done to ensure only cities who have trained are considered. 
    df_prediction    = df_prediction.merge(area_df_pred, on=['adm2_name', 'adm1_name'], how='inner') # done to ensure only cities who have trained are considered. 

    df_prediction["predicted_value"] = np.nan
    df_prediction,corn_yield_tot    = RF_prediction(model_dict,df_prediction,features)

    print(corn_yield_tot)

    return df_prediction, corn_yield_tot


Now we insert 2024 (or whatever) data on which we want to do the forecast. The output of the YEAR_PREDICTION corn yield forecast is shown in the plot below together with its standard deviation.

In [ ]:
if RANDOM_FOREST:
    df_prediction, corn_yield_tot = prediction(feat_year,YEAR_PREDICTION,features)
    df            = download_corn_data()
    fig, ax = plt.subplots(ncols=2,nrows=1,figsize=(8,3))
    df.plot(x="year",y="Value",ax=ax[0])
    ax[0].errorbar(x=[YEAR_PREDICTION],y=[corn_yield_tot], yerr=[std_RF],  fmt="*",  capsize=5, label="uncertanty" )
    ax[0].legend(["production", "prevision and uncertanty"])

    df_prediction_2012, corn_yield_tot_2012 = prediction(feat_year,2012,features)
    df.plot(x="year",y="Value",ax=ax[1])
    ax[1].errorbar(x=[2012],y=[corn_yield_tot_2012], yerr=[std_RF],  fmt="*",  capsize=5, label="uncertanty" )
    ax[1].legend(["production", "prevision and uncertanty"])

The cities who have not been trained are not predicted since there is no trend available and we have seen that the trend is crucial parameter thus we do not know which would be the mean value but we could only know the fluctuations

In the following we create a fucntion to check where two dataframes have different counties and then we check if all couties whose corn yield was available in 2023 have also been forecast for 2024

In [ ]:
# Given df1 and df2 return a dataframe with elements "adm1_name", "adm2_name" that are present in df1 but not in df2
def missing_groups(df1, df2):
    missing_gr_df = df1.merge(df2[["adm1_name", "adm2_name"]], on=["adm1_name", "adm2_name"],
        how="left",indicator=True).query('_merge == "left_only"')[["adm1_name", "adm2_name"]]
    return missing_gr_df

The next cell acts on the dataframe: it expand the dataframe so that each raw is an input for training (it includes 4 times more input parameters than before because it includes the data of the available month- January to April)

In [ ]:
def expand_month_features_new(df):

    if features_NN_training == "all":
        features_wanted = extract_feature_list(feat_year)
    else:
        features_wanted = features_NN_training
        
    if ("month_N" in df):
        mask = df["month_N"].between(*MONTHS_RANGE)
        df = df[mask]
            
        if "month_N" in features_wanted:
            features_wanted.remove("month_N")

        list_to_keep = [el for el in df.columns if not(el in features_wanted) and not("month" in el) ]

        print(list_to_keep)
        df_wide = df.pivot(index=list_to_keep, columns="month_N", values=features_wanted )

        all_features        = [f"{var}_m{month}" for var, month in df_wide.columns]

        df_wide.columns = [f"{var}_m{month}" for var, month in df_wide.columns]

        df_wide = df_wide.reset_index()

        drop_list = [feat for feat in all_features if df_wide[feat].std() == 0]
        features  = [feat for feat in all_features if not(df_wide[feat].std() == 0)]

        df_wide.drop(drop_list, axis=1,inplace=True)
        
        return df_wide, features
    else:
        return df, features_wanted

In [ ]:
if RANDOM_FOREST:
    train_df, features                = expand_month_features_new(train_yield_feat_month)
    rmse_total, model_dict, std_RF    = random_forest_training(train_df, features, plot_y_test=False)


Now we do again the prediction with the random forest with the model trained on the monthly aggregate feature

In [ ]:
if RANDOM_FOREST:
    df_pred, features             =  expand_month_features_new(feat_month)
    df_prediction, corn_yield_tot = prediction(df_pred,YEAR_PREDICTION,features)

    df       = download_corn_data()
    fig, ax  = plt.subplots(ncols=2,nrows=1,figsize=(7,3))
    df.plot(x="year",y="Value",ax=ax[0])
    df.plot(x="year",y="Value",ax=ax[1])

    ax[0].errorbar(x=[YEAR_PREDICTION],y=[corn_yield_tot], yerr=[std_RF],  fmt="*",  capsize=5, label="uncertanty" )
    ax[0].legend(["production", "prevision and uncertanty"])
    ax[1].errorbar(x=[YEAR_PREDICTION],y=[corn_yield_tot], yerr=[std_RF],  fmt="*",  capsize=5, label="uncertanty" )
    ax[1].legend(["production", "prevision and uncertanty"])
    ax[1].set_xlim([2000,2025])
    print("standard deviation is: "+str(std_RF))


We can see that when trained on the enlarged dataset (that includes also data in different month and not just the yearly aggregated features) the prediction goes up from 190 to 195.

At this point we look for a more sophisticated model for the production. We are going to use deep-learning. 

In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

has_mps = torch.backends.mps.is_built()
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu" # gpu not tested
print(f"Using device: {device}")

model_NN = ["MLP_model","prob_MLP_model"]  # approach removed["LSTM_model"]

model_NN = model_NN[0]

yearly_normalization = True

with_state_in_the_model = False
with_embedding          = False

months_range = list(MONTHS_RANGE) # for compatibility with other functions. It should be cleaned.



Here we prepare the dataset for the training and once again we expand our dataset so that each raw is an input with many features per month (there are 4 month January to April)

In [ ]:
if features_NN_training == "all":
    features_wanted = extract_feature_list(feat_month)
else:
    features_wanted = features_NN_training


mask = (train_yield_feat_month["year"] < YEAR_PREDICTION) & (train_yield_feat_month["month_N"].between(*months_range))

df = train_yield_feat_month[mask]
if "month_N" in features_wanted:
    features_wanted.remove("month_N")

df_wide = df.pivot(index=["adm2_name", "adm1_name", "year",'Value','Area_prediction', 'Value-detrand'], columns="month_N", values=features_wanted )

features_wanted = [f"{var}_m{month}" for var, month in df_wide.columns]
df_wide.columns = [f"{var}_m{month}" for var, month in df_wide.columns]



df_wide = df_wide.reset_index()

df_train, df_test = train_test_split(df_wide, test_size=0.2, random_state=0)

features = []
for ff in features_wanted:
    if ff in df_train.columns:
        if not(df_train[ff].std() == 0):
            features.append(ff)
print("using the following features: ")
print(", ".join(features))


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train['state_idx'] = le.fit_transform(df_train['adm1_name'])  # OK
df_test['state_idx'] = le.fit_transform(df_test['adm1_name'])  # OK


Now we create the right input format for the Pytorch API (the .to(device) call can be cleaned and removed because I could not test it on gpu so it is forced to run on cpu) 

In [ ]:
labels     = ['Value-detrand']
group_cols = ['adm2_name',"adm1_name",'year']
# Create empty list for sequences and keys
X_train = []
Y_train = []
X_test  = []
Y_test  = []
keys_train = []
keys_test  = []

# Calcolo media e std solo su train
means = df_train[features].mean().to_numpy()
stds  = df_train[features].std().to_numpy()

# Normalizzazione
X_train = (df_train[features].to_numpy() - means) / stds
X_test  = (df_test[features].to_numpy()  - means) / stds

Y_train = df_train[labels].to_numpy()
Y_test  = df_test[labels].to_numpy()



if with_state_in_the_model and not(with_embedding):
    # one-hot encode state
    state_oh_train = pd.get_dummies(df_train['state_idx']).values.astype('float32')
    state_oh_test = pd.get_dummies(df_test['state_idx']).reindex(columns=range(len(states)), fill_value=0).values.astype('float32')

    # concat weather + state
    X_train = np.concatenate([X_train, state_oh_train], axis=1)
    X_test = np.concatenate([X_test, state_oh_test], axis=1)

    add_dim = 13
else:
    add_dim = 0 

x_train = torch.tensor(X_train, dtype=torch.float32).view(-1, len(features)+add_dim)
y_train = torch.tensor(Y_train, dtype=torch.float32).view(-1, 1)
x_test  = torch.tensor(X_test, dtype=torch.float32).view(-1, len(features)+add_dim)
y_test  = torch.tensor(Y_test, dtype=torch.float32).view(-1, 1)

if with_embedding:

    state_train = torch.tensor(df_train['state_idx'].to_numpy(), dtype=torch.long)
    state_test  = torch.tensor(df_test['state_idx'].to_numpy(), dtype=torch.long)
    train_dataset = TensorDataset(x_train, state_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(x_test, state_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

else:

    # Setup data loaders for batch
    train_dataset = TensorDataset(x_train, y_train)
    print(x_train.shape)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    test_dataset = TensorDataset(x_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Here we build the NN (actually I started implementing a second one but I did not finish)

In [ ]:
import torch.nn.functional as F
    
class MLPModel_old(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x.squeeze(1)  #
    
class MLPModel(nn.Module):
    def __init__(self, input_dim, num_states=0, emb_dim=0):
        super().__init__()
        self.emb_dim = emb_dim  # salva per uso nel forward

        if self.emb_dim > 0:
            self.state_emb = nn.Embedding(num_states, emb_dim)
            input_dim = input_dim + emb_dim
        else:
            input_dim = input_dim

        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 1)

    def forward(self, weather_x, state_idx=None):
        if self.emb_dim > 0:
            state_embed = self.state_emb(state_idx)
            x = torch.cat([weather_x, state_embed], dim=1)
        else:
            x = weather_x

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x.squeeze(1)

if with_embedding:
    model = MLPModel(input_dim=x_train.shape[1], num_states=len(states), emb_dim=4).to(device)
else:
    model = MLPModel(input_dim=x_train.shape[1]).to(device)

model

Here we train the model and evaluate the loss on the validation dataset to check if the network is learning or overfitting.

We print also the variance of the test set and the variance of the predicted output to be sure that we are not flattening aroung a single mean value

In [ ]:
# Train the model
criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=3)

epochs = 30
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        if with_embedding:
            x_batch, state_batch, y_batch = batch
            outputs = model(x_batch,state_batch)
        else:
            x_batch, y_batch = batch
            outputs = model(x_batch)

        y_batch = y_batch.squeeze(-1)  #  for dimension compatibility
        loss = criterion(outputs, y_batch)

        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for idx, batch in enumerate(test_loader):
            if with_embedding:
                x_batch, state_batch, y_batch = batch
                outputs = model(x_batch,state_batch)
            else:
                x_batch, y_batch = batch
                outputs = model(x_batch)
            y_batch = y_batch.squeeze(-1)  # for dimension compatibility
            loss = criterion(outputs, y_batch)
                
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 20:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.8f}")

    print("Batch output variance:", torch.var(outputs))
    print("Batch labels variance:", torch.var(y_batch))



Now we plot training and test dataset versus the corresponding output generated by the network to actually check if the NN has learned or not

In [ ]:
model.eval()
with torch.no_grad():
    if with_embedding:
        y_pred_train = model(x_train.to(device),state_train).cpu().numpy()
        y_pred_test  = model(x_test.to(device),state_test).cpu().numpy()
    else:
        y_pred_train = model(x_train.to(device)).cpu().numpy()
        y_pred_test  = model(x_test.to(device)).cpu().numpy()

    y_train_np   = y_train.cpu().numpy()
    y_test_np    = y_test.cpu().numpy()
    
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,4))
ax[1].scatter(y_train_np, y_pred_train, s=5, alpha=0.5)
ax[1].set_xlabel("True y (train)")
ax[1].set_ylabel("Predicted y")
ax[1].set_title("Train fit")
ax[1].plot([y_train_np.min(), y_train_np.max()], [y_train_np.min(), y_train_np.max()], 'r--')
ax[1].set_xlim([-2,1])

ax[0].scatter(y_test_np, y_pred_test, s=5, alpha=0.5)
ax[0].set_xlabel("True y (test)")
ax[0].set_ylabel("Predicted y")
ax[0].set_title("Test fit")
ax[0].plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], 'r--')

ax[0].set_xlim([-2,1])

Uncertanty quantification:
Now we estimate the prediction uncertainty using conformal prediction, we compute the absolute errors on the test set and finds the quantile such that only an "alpha" fraction exceed it

In [ ]:
alpha = 0.05  # 95% coverage
model.eval()
err_list = []
with torch.no_grad():
    for batch in test_loader:
        if with_embedding:        
            x_test_batch, state_test_batch, y_test_batch = batch
            x_test_batch = x_test_batch.to(device)
            state_test_batch = state_test_batch.to(device)
            y_test_batch = y_test_batch.to(device).squeeze(-1)
            y_pred_test = model(x_test_batch, state_test_batch)
        else:
            x_test_batch, y_test_batch = batch
            x_test_batch = x_test_batch.to(device)
            y_test_batch = y_test_batch.to(device).squeeze(-1)
            y_pred_test = model(x_test_batch)
        err         = torch.abs(y_test_batch - y_pred_test)
        err_list.append(err.cpu())

err_list = torch.cat(err_list, dim=0).numpy()
N_test   = err_list.shape[0]

q_level = np.ceil((N_test + 1) * (1 - alpha)) / N_test # useless correction Ntest (used for accounting finite samples)
q_cal   = np.quantile(err_list, q_level, method="higher")

print("Estimate for uncertenty: "+str((1-alpha)*100)+"% of the predictions fall within "+str(q_cal)+ " of the true values -for NN y-true values dataset-")

Finally we apply the NN to the weather dataset of 2024 (or of YEAR_PREDICTION) to get the prediction

In [ ]:
model.eval()
with torch.no_grad():
    df_prediction_NN    = feat_month[feat_month["year"]==YEAR_PREDICTION]
    corn_stats          = (corn_value_tot.drop_duplicates(subset=["adm1_name", "adm2_name"])[["adm1_name", "adm2_name", "mean", "std"]])
    df_prediction_NN    = df_prediction_NN.merge(corn_stats, on=['adm2_name', 'adm1_name'], how='inner') # done to ensure only cities who have trained are considered. 
    df_prediction_NN    = df_prediction_NN.merge(area_df_pred, on=['adm2_name', 'adm1_name'], how='inner') # done to ensure only cities who have trained are considered. 

    features_total = extract_feature_list(feat_month)

    df_wide = df_prediction_NN.pivot(index=["adm2_name", "adm1_name", "year","Area_prediction","std","mean"], columns="month_N", values=features_total )

    df_wide.columns = [f"{var}_m{month}" for var, month in df_wide.columns]

    df_wide = df_wide.reset_index()

    means = df_train[features].mean().to_numpy()
    stds  = df_train[features].std().to_numpy()

    area_YEAR_PREDICTION   = df_wide["Area_prediction"]
    state_input            = df_wide["adm1_name"].to_numpy()

    if with_embedding:
        le = LabelEncoder()
        df_wide['state_idx'] = le.fit_transform(df_wide['adm1_name'])  # OK
        state_input_pred     = torch.tensor(df_wide['state_idx'].to_numpy(), dtype=torch.long)

    X_YEAR_PREDICTION      = (df_wide[features].to_numpy() - means) / stds
    x_YEAR_PREDICTION      = torch.tensor(X_YEAR_PREDICTION, dtype=torch.float32).view(-1, len(features))   

    if with_embedding:
        y_pred_YEAR_PREDICTION = model(x_YEAR_PREDICTION.to(device), state_input_pred).cpu().numpy()
    else:
        y_pred_YEAR_PREDICTION = model(x_YEAR_PREDICTION.to(device)).cpu().numpy()

    range_err   = torch.as_tensor(q_cal).numpy()

    corn_yield_tot = []
    # this value must be retrended
    for val in [y_pred_YEAR_PREDICTION, y_pred_YEAR_PREDICTION + range_err, y_pred_YEAR_PREDICTION - range_err]:
        df_wide["Value_predicted"] = val

        pred_df = restore_trend(df_wide,model_trend_dict,log_detrend=log_transf, trend_detrend=linear_detrand, normalize_data_mean_std=mean_std_norm)


        total_production = np.sum(pred_df["Value_restored"].to_numpy() * pred_df["Area_prediction"].to_numpy())
        total_area       = np.sum(pred_df["Area_prediction"].to_numpy())

        print(total_production/total_area)

        corn_yield_tot.append(total_production/total_area )

    if with_embedding:
        pred_w_embedding  = corn_yield_tot
    else:
        pred_wo_embedding = corn_yield_tot


#print(pred_w_embedding, pred_wo_embedding)

We plot the predicted value and the interval of uncertanty

In [ ]:
df       = download_corn_data()
fig, ax  = plt.subplots(ncols=1,nrows=1,figsize=(4,3))
df.plot(x="year",y="Value",ax=ax)

ax.errorbar(x=[YEAR_PREDICTION],y=[corn_yield_tot[0]], yerr=[corn_yield_tot[1]-corn_yield_tot[0]],  fmt="*",  capsize=5, label="uncertanty" )
ax.legend(["production", "prevision and uncertanty"])

print("The predicted value is: "+str(corn_yield_tot[0]))
print("Uupper and lower boundary: " +str(corn_yield_tot[1]) +" and "+ str(corn_yield_tot[2]))
# print("Estimate for uncertenty: "+str((1-alpha)*100)+"% of the predictions fall within "+str(total_production[2])+ " of the true values -for NN y-true values dataset-")